## Real-Time Race, Age, and Gender Prediction
This notebook showcases an innovative approach to predicting an individual's race, age, and gender in real-time using live video feed. Leveraging advanced computer vision and deep learning techniques, our solution processes video input to detect faces and then predicts demographic information with remarkable accuracy.

In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load pre-trained models
age_model_path = 'saved_models_age/epoch-20-val_loss-0.0038-val-0.0056.h5'
gender_model_path = 'saved_models_gender/-27---0.2106-0.9443---0.1833-0.9289.h5'
race_model_path = 'saved_models_race/-06---0.5111-0.8378---0.4894-0.8366.h5'

age_model = load_model(age_model_path)
gender_model = load_model(gender_model_path)
race_model = load_model(race_model_path)

In [ ]:
# Initialize the face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_input_vggface(x):
    """Preprocess the input to match the VGGFace model's expected format."""
    x = np.array(x, dtype=np.float32)
    x[..., 0] -= 93.5940
    x[..., 1] -= 104.7624
    x[..., 2] -= 129.1863
    return x


cap = cv2.VideoCapture(1)  # Adjust the index based on your camera setup

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face_array = preprocess_input_vggface(face)
        face_array_expanded = np.expand_dims(face_array, axis=0)
        
        predicted_age = age_model.predict(face_array_expanded)[0][0] * 116
        gender_prediction = gender_model.predict(face_array_expanded)
        gender = 'Male' if gender_prediction[0][0] < 0.5 else 'Female'
        
        race_prediction = race_model.predict(face_array_expanded)
        race_index = np.argmax(race_prediction[0])
        races = ['White', 'Black', 'Asian', 'Indian', 'Others']
        race = races[race_index]
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'Age: {int(predicted_age)}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.putText(frame, f'Gender: {gender}', (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.putText(frame, f'Race: {race}', (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Age, Gender, and Race Prediction', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 32ms/step
